In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from torch_geometric.utils import softmax
import torch_geometric.utils.convert as conv
import torch
from torch_geometric.nn import GCNConv, GATConv,SAGEConv
import torch.nn.functional as F
from torch.nn.functional import softmax

# load the graph from the graphml file
def load_graph(file_path):
    G = nx.read_graphml(file_path)
    return G

file_path = 'data/citeseer.graphml'
# define the classification label
classification_label = 'subject'


In [2]:
G = load_graph(file_path)

In [6]:
#Removing not useful attributes
for s,d in G.edges:
    del G.edges[s,d]['label']

In [4]:
print(G, type(G))

print(list(G.nodes(data=True))[0])


DiGraph with 3312 nodes and 4715 edges <class 'networkx.classes.digraph.DiGraph'>
('100157', {'w_0': 0, 'w_1': 0, 'w_2': 0, 'w_3': 0, 'w_4': 0, 'w_5': 0, 'w_6': 0, 'w_7': 0, 'w_8': 0, 'w_9': 0, 'w_10': 0, 'w_11': 0, 'w_12': 0, 'w_13': 0, 'w_14': 0, 'w_15': 0, 'w_16': 0, 'w_17': 0, 'w_18': 0, 'w_19': 0, 'w_20': 0, 'w_21': 0, 'w_22': 0, 'w_23': 0, 'w_24': 0, 'w_25': 0, 'w_26': 0, 'w_27': 0, 'w_28': 0, 'w_29': 0, 'w_30': 0, 'w_31': 0, 'w_32': 0, 'w_33': 0, 'w_34': 0, 'w_35': 0, 'w_36': 1, 'w_37': 0, 'w_38': 0, 'w_39': 0, 'w_40': 0, 'w_41': 0, 'w_42': 0, 'w_43': 0, 'w_44': 0, 'w_45': 0, 'w_46': 1, 'w_47': 0, 'w_48': 0, 'w_49': 0, 'w_50': 0, 'w_51': 0, 'w_52': 0, 'w_53': 0, 'w_54': 0, 'w_55': 0, 'w_56': 0, 'w_57': 0, 'w_58': 0, 'w_59': 0, 'w_60': 0, 'w_61': 0, 'w_62': 0, 'w_63': 0, 'w_64': 0, 'w_65': 1, 'w_66': 0, 'w_67': 0, 'w_68': 0, 'w_69': 0, 'w_70': 0, 'w_71': 0, 'w_72': 0, 'w_73': 0, 'w_74': 0, 'w_75': 0, 'w_76': 0, 'w_77': 0, 'w_78': 0, 'w_79': 0, 'w_80': 0, 'w_81': 0, 'w_82': 0, 'w_

In [7]:
edge = list(G.edges(data=True))[0]

print(edge)


('100157', '100157', {})


Convert the node attributes into integers

In [12]:
classes = set()
for id in G.nodes:
    classes.add(G.nodes[id]['subject'])

labels = {'AI':0, 'HCI':1, 'Agents':2, 'IR':3, 'DB':4, 'ML':5,
          0:0,1:1,2:2,3:3,4:4,5:5}
for id in G.nodes:
    G.nodes[id]['subject'] = labels[G.nodes[id]['subject']]

g_node_attrs = list(list(G.nodes(data=True))[0][1].keys())

print(g_node_attrs)



num_classes = len(classes)

print(classes)

w_attrs = [attr for attr in g_node_attrs if 'w_' in attr]

print(len(g_node_attrs))

['w_0', 'w_1', 'w_2', 'w_3', 'w_4', 'w_5', 'w_6', 'w_7', 'w_8', 'w_9', 'w_10', 'w_11', 'w_12', 'w_13', 'w_14', 'w_15', 'w_16', 'w_17', 'w_18', 'w_19', 'w_20', 'w_21', 'w_22', 'w_23', 'w_24', 'w_25', 'w_26', 'w_27', 'w_28', 'w_29', 'w_30', 'w_31', 'w_32', 'w_33', 'w_34', 'w_35', 'w_36', 'w_37', 'w_38', 'w_39', 'w_40', 'w_41', 'w_42', 'w_43', 'w_44', 'w_45', 'w_46', 'w_47', 'w_48', 'w_49', 'w_50', 'w_51', 'w_52', 'w_53', 'w_54', 'w_55', 'w_56', 'w_57', 'w_58', 'w_59', 'w_60', 'w_61', 'w_62', 'w_63', 'w_64', 'w_65', 'w_66', 'w_67', 'w_68', 'w_69', 'w_70', 'w_71', 'w_72', 'w_73', 'w_74', 'w_75', 'w_76', 'w_77', 'w_78', 'w_79', 'w_80', 'w_81', 'w_82', 'w_83', 'w_84', 'w_85', 'w_86', 'w_87', 'w_88', 'w_89', 'w_90', 'w_91', 'w_92', 'w_93', 'w_94', 'w_95', 'w_96', 'w_97', 'w_98', 'w_99', 'w_100', 'w_101', 'w_102', 'w_103', 'w_104', 'w_105', 'w_106', 'w_107', 'w_108', 'w_109', 'w_110', 'w_111', 'w_112', 'w_113', 'w_114', 'w_115', 'w_116', 'w_117', 'w_118', 'w_119', 'w_120', 'w_121', 'w_122', 'w

In [14]:
# delete all the graph attributes, since they are not useful for node classification

# print(type(node_attrs))
# print(edge_attrs)

# graph.graph['node_default'] = True
# graph.graph['edge_default'] = True

print(G.graph, type(G.graph))

if 'node_default' in G.graph:
    del G.graph['node_default']
if 'edge_default' in G.graph:
    del G.graph['edge_default']

if 'edge_index' in G.graph:
    del G.graph['edge_index']

attrs = [attr for attr, _ in G.graph.items()]

for attr in attrs:
    del G.graph[attr]

print(G.graph, type(G.graph))

{} <class 'dict'>
{} <class 'dict'>


In [28]:
data = conv.from_networkx(G, group_node_attrs=g_node_attrs)

print(f'Number of edges: {data.num_edges}')

print(data, type(data))

Number of edges: 4715
Data(edge_index=[2, 4715], x=[3312, 3704]) <class 'torch_geometric.data.data.Data'>


Extract the classification label from the nodes attributes

In [29]:
print(data.x)

# shuffle the data by row
data.x = data.x[torch.randperm(data.x.size()[0])]
print(data.x.shape)

# get the classification attibute from x and assign it to y
data.y = data.x[:, -1]
# data.y = torch.stack([y[0] for y in data.y])

# remove the classification attribute from x
data.x = data.x[:, :-1].float()

print(data.x)
print(data.y)

print(data)

tensor([[0, 0, 0,  ..., 0, 0, 2],
        [0, 0, 0,  ..., 0, 0, 2],
        [0, 0, 0,  ..., 0, 0, 2],
        ...,
        [0, 0, 0,  ..., 0, 0, 5],
        [0, 0, 0,  ..., 0, 0, 3],
        [0, 0, 0,  ..., 0, 0, 1]])
torch.Size([3312, 3704])
tensor([[0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([2, 3, 4,  ..., 2, 5, 5])
Data(edge_index=[2, 4715], x=[3312, 3703], y=[3312])


Define training, validation and test sets randomly

In [30]:
from torch_geometric.transforms import RandomNodeSplit

# split the data in train, validation and test sets
# it adds to data train, val and test masks
rns = RandomNodeSplit('random', num_val=0.1, num_test=0.2)

data = rns(data)

print(data)

Data(edge_index=[2, 4715], x=[3312, 3703], y=[3312], train_mask=[3312], val_mask=[3312], test_mask=[3312])


One-hot encode the classification labels

In [31]:
# one hot encode labels
data.y = torch.nn.functional.one_hot(data.y.long(), num_classes=num_classes).to(torch.float)

print(data.y)
print(data)

tensor([[0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]])
Data(edge_index=[2, 4715], x=[3312, 3703], y=[3312, 6], train_mask=[3312], val_mask=[3312], test_mask=[3312])


In [32]:
print(data.x.dtype)
print(data.y.dtype)
print(data.edge_index.dtype)

torch.float32
torch.float32
torch.int64


In [33]:
print(list(data.edge_index))

[tensor([   0,    1,    2,  ..., 3307, 3309, 3311]), tensor([   0,    0,    0,  ..., 3308,  479, 2871])]


In [34]:
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [40]:
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(input_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        # x = softmax(x)
        # x = F.log_softmax(x, dim=1)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = GCN(input_features=in_features, hidden_channels=16)
model.eval()
print(model)

out = model(data.x, data.edge_index)
#visualize(out, color=data.y)
print(out)

model = GCN(input_features=in_features, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


GCN(
  (conv1): GCNConv(3703, 16)
  (conv2): GCNConv(16, 6)
)
tensor([[0.1570, 0.1747, 0.1564, 0.1702, 0.1853, 0.1564],
        [0.1672, 0.1631, 0.1631, 0.1765, 0.1669, 0.1631],
        [0.1721, 0.1625, 0.1616, 0.1786, 0.1616, 0.1635],
        ...,
        [0.1650, 0.1650, 0.1650, 0.1752, 0.1650, 0.1650],
        [0.1662, 0.1660, 0.1625, 0.1676, 0.1678, 0.1699],
        [0.1552, 0.1748, 0.1670, 0.1567, 0.1910, 0.1552]],
       grad_fn=<SoftmaxBackward0>)


In [63]:
class SAGE(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = SAGEConv(input_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = SAGE(input_features=in_features, hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [45]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)

        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        self.conv1 = GATConv(in_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)


    def forward(self, x, edge_index):

        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.softmax(x, dim=1)

model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

GAT(
  (conv1): GATConv(3703, 8, heads=8)
  (conv2): GATConv(64, 6, heads=1)
)


In [64]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      val_loss = criterion(out[data.val_mask], data.y[data.val_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss, val_loss


def test(models, ds='test'):
      models.eval()
      out = models(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      if ds == 'test':
            outs = out[data.test_mask]
            mask = data.test_mask
            d_set = pred[data.test_mask]
            label_set = data.y[data.test_mask]
      elif ds == 'val':
            outs = out[data.val_mask]
            mask = data.val_mask
            d_set = pred[data.val_mask]
            label_set = data.y[data.val_mask]
      else:
            outs = out[data.train_mask]
            mask = data.train_mask
            d_set = pred[data.train_mask]
            label_set = data.y[data.train_mask]
      # print(outs)
      # print(label_set)
      label_set = label_set.argmax(dim=1)
      test_correct = d_set == label_set  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return test_acc

In [65]:
start_patience = 40
patience = start_patience
best_val_loss = 10000
best_model = model

for epoch in range(1, 101):
    if patience == 0:
        break
    loss, val_loss = train()
    if val_loss <= best_val_loss:
        patience = start_patience
        best_val_loss = val_loss
        best_model = model
    else:
        patience -= 1

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')


train_acc = test(best_model, 'train')
val_acc = test(best_model, 'val')
test_acc = test(best_model)

print(f'Train Accuracy: {train_acc:.4f}')
print(f'Val Accuracy: {val_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

model.eval()

# out = model(data.x, data.edge_index)
# visualize(out, color=data.y)

Epoch: 001, Loss: 1.7910, Val Loss: 1.7921
Epoch: 002, Loss: 1.6207, Val Loss: 1.7729
Epoch: 003, Loss: 1.5868, Val Loss: 1.7675
Epoch: 004, Loss: 1.5216, Val Loss: 1.7701
Epoch: 005, Loss: 1.4887, Val Loss: 1.7410
Epoch: 006, Loss: 1.4782, Val Loss: 1.7760
Epoch: 007, Loss: 1.4249, Val Loss: 1.7044
Epoch: 008, Loss: 1.4740, Val Loss: 1.6823
Epoch: 009, Loss: 1.4274, Val Loss: 1.6457
Epoch: 010, Loss: 1.4388, Val Loss: 1.6835
Epoch: 011, Loss: 1.3661, Val Loss: 1.6922
Epoch: 012, Loss: 1.3647, Val Loss: 1.7231
Epoch: 013, Loss: 1.3724, Val Loss: 1.7107
Epoch: 014, Loss: 1.3632, Val Loss: 1.6937
Epoch: 015, Loss: 1.3614, Val Loss: 1.6804
Epoch: 016, Loss: 1.3936, Val Loss: 1.6705
Epoch: 017, Loss: 1.3812, Val Loss: 1.6456
Epoch: 018, Loss: 1.3629, Val Loss: 1.6348
Epoch: 019, Loss: 1.4041, Val Loss: 1.6684
Epoch: 020, Loss: 1.3545, Val Loss: 1.6616
Epoch: 021, Loss: 1.3722, Val Loss: 1.6814
Epoch: 022, Loss: 1.3238, Val Loss: 1.6353
Epoch: 023, Loss: 1.3689, Val Loss: 1.6675
Epoch: 024,

SAGE(
  (conv1): SAGEConv(3703, 64, aggr=mean)
  (conv2): SAGEConv(64, 6, aggr=mean)
)